# 火币基础数据收集-异常处理(Huobi Candle Collecting-Exception Handling)

In [ ]:
from huobi import *
from huobi.model import *

import json
import time
import pandas as pd
import datetime

In [ ]:
import mysql.connector as db

db_config = {
    "database": "Huobi_Ex",
    "user": "root",
    "password": "root",
    "host": "localhost",
    "port": 3306,
}

# 连接数据库获取游标，可以设置返回数据的格式，元组，命令元组，字典等...
connect = db.connect(**db_config)

In [ ]:
#subscription_client = SubscriptionClient(api_key="*****************", secret_key="*****************")

subscription_client = SubscriptionClient()
subscription_client.unsubscribe_all()

def callback(candlestick_event: 'CandlestickEvent'):
    sqli = "replace into `kline_%(symbol)s_%(period)s` (`amount`, `open`, `close`, `high`, `id`, `count`, `low`, `vol`) \
        select %(amount)s, %(open)s, %(close)s, %(high)s, %(id)s, %(count)s, %(low)s, %(vol)s where \
        not exists(select id from `kline_%(symbol)s_%(period)s` where id = %(id)s and amount = %(amount)s) "
    
    #print(candlestick_event.symbol)
    #print(candlestick_event.interval)
    item = {}
    i = 0
    j = 0
    
    for candlestick in candlestick_event.data:
        item.update({"symbol":candlestick_event.symbol,\
                     "period":candlestick_event.interval,\
                     "id":candlestick.id,\
                     "open":candlestick.open,\
                     "close":candlestick.close,\
                     "low":candlestick.low,\
                     "high":candlestick.high,\
                     "amount":candlestick.amount,\
                     "count":candlestick.count,\
                     "vol":candlestick.volume,\
                    })
        sqlcmd = sqli%(item)
        #print(sqlcmd)
        cursor.execute(sqlcmd)
    
        if cursor.rowcount == 2 :
            j += 1
        else :
            i+=cursor.rowcount
        
        #print(cursor.rowcount)
        i += j//2
    
    connect.commit()
    print("keypair:%s :: period:%s,  rows:%s, effect rows:%s, update rows:%s || %s"%(candlestick_event.symbol.rjust(10,' '), candlestick_event.interval.rjust(8,' '), str(len(candlestick_event.data)).rjust(5,' '), str(i).rjust(5,' '), str(j).rjust(5,' '),  item["id"]))


In [ ]:
endtimeflag = datetime.date(2020, 5, 3)
print(int(time.mktime(endtimeflag.timetuple())))

In [ ]:
#symbol: 'btcusdt', 'ethusdt', 'htusdt', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', \
#         'bsvusdt', 'trxusdt', 'zecusdt', 'dashusdt', 'adausdt', 'iotausdt', 'xmrusdt', 'hb10usdt', \
#         'newusdt', 'seeleusdt', 'wiccusdt', 'ontusdt', 'mxusdt', 'hptusdt', 'ektusdt', 'gtusdt', \
#         'lambusdt', 'bixusdt', 'linkusdt', 'algousdt', 'wavesusdt', 'vidyusdt', 'atomusdt', 'iostusdt', 'qtumusdt'

#interval: 1min, 5min, 15min, 30min, 60min, 4hour, 1day, 1mon, 1week, 1year
#from : [1501174800, 2556115200]8
#to :[1501174800, 2556115200]
#1582500000 : Timestamp('2020-02-24 07:20:00')
#size : (0,300]

periods = ['1min', '5min', '15min', '30min', '60min', '4hour']
# symbol =  ['btcusdt', 'ethusdt', 'htusdt', 'htbtc', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', \
#          'bsvusdt', 'trxusdt', 'xmrusdt', 'hb10usdt', 'linkusdt', 'zecusdt', 'dashusdt', 'adausdt', 'iotausdt', \
#          'newusdt', 'seeleusdt', 'wiccusdt', 'ontusdt', 'mxusdt', 'hptusdt', 'ektusdt', 'gtusdt', \
#          'lambusdt', 'bixusdt', 'algousdt', 'wavesusdt', 'vidyusdt', 'atomusdt', 'iostusdt', 'qtumusdt']
symbol =  ['htbtc', 'ethbtc', 'etcbtc', 'bsvbtc', 'linkbtc', 'xrpbtc', 'ltcbtc', 'bchbtc', 'eosbtc']
timestep = [60, 300, 900, 1800, 3600, 14400] 
step = 300
fromtime = 1501387200
endtime = 1582500000

# 1508990400 Timestamp('2017-10-26 12:00:00')

In [ ]:
n=8
k=0

cursor = connect.cursor()

startpoint = 1580934400
errorlist =range(0,10, 1) #[121, 220, 246, 249, 440, 459, 475, 533, 682, ] #range(69, 101, 1) #1571062920

for item in errorlist: 
    fromtime = startpoint + step * timestep[k] * (item -1) #1530783200
    subscription_client.request_candlestick_event(symbol[n], periods[k], callback, from_ts_second=fromtime,end_ts_second=fromtime + step * timestep[k], auto_close=True, error_handler=None) #1582500000
    time.sleep(5)

time.sleep(30)
cursor.close()